In [28]:
# If you are viewing this as a python script comment out these lines.
%cd /
%cd '/content/drive/My Drive/HCL hiring challenge'
%cd HCL\ ML\ Challenge

/
/content/drive/My Drive/HCL hiring challenge


In [29]:
import pandas as pd
import re
import os
import decimal
from tqdm import tqdm
import json
import html

/content/drive/My Drive/HCL hiring challenge/HCL ML Challenge


In [31]:
# List all the files 
os.chdir(f'HCL ML Challenge Dataset')
list_dir = os.listdir()
print(list_dir[:5]) #print first five records

['X8XUG1JD.txt',
 'X8XX000W.txt',
 'X8XX00J6.txt',
 'X8XX00LL.txt',
 'X8XX00YW.txt']

In [6]:
# save the names of all files in a list called names.
names=[i.split('.')[0] for i in list_dir]
print(names[:5]) #print first five records

['X8XUG1JD', 'X8XX000W', 'X8XX00J6', 'X8XX00LL', 'X8XX00YW']

In [0]:
def check_para(each_line):
  """
  This function basically checks for numbers with paranthesis and replaces 
  it with the  negative of the number as perthe requirements of the challenge.
  e.g : (11345) ==> -11345
  
  Credits: https://www.quora.com/How-can-I-convert-a-number-given-in-a-parenthesis-with-a-negative-number-in-Python
  """
  num=0
  # we don't need empty values, we only need to check for numbers. 
  if each_line == 'nan':
    return 'nan'
  # this pattern matches numbers in brackets.
  pat = re.compile(r'(\(?(\d+)\)?)')
  this = pat.search(each_line)
  if this is not None:
    num = decimal.Decimal(this.groups()[1])
    if this.groups()[0].startswith('('):
      num *= -1
  return str(int(num))

Text approach

In [148]:
# this is the main for loop that extracts the values from the 500 files.
values = []
for i in tqdm(names[:],position=0,leave=True):

  # Create an I/O object and read the data from the file.

  temp = open('{}.txt'.format(i),'r')
  temp1 = temp.readlines()
  temp.close()

  # close the file object to avoid any exceptions.

  # Create lists to that will be used to create  a dataframe.

  l=[]
  l1 = []
  l2 = []
  l3 = []  

  # remove whitespaces
  for i in temp1:
    l.append(i.strip())

  # split lines based on the gap between them.
  for i in l:
    l1.append(i.split('  '))

  # filter out the None values and append
  for i in l1:
    l2.append(filter(None,i))
  
  # create lists of the seperated columns.
  for i in l2:
    l3.append(list(i))
  
  # Remove the header of the files which aren't required for this task.
  l3.remove(l3[0])

  # we don't need the  unnecessary satements part below the balance sheet.(this number is set after fine-tuning)
  l3 = l3[:20]


  try:
    # add the header for the first column if it isn't present.
    if l3[0][0] != 'Notes': 
      l3[0].insert(0,'nan')

    # Try : Except block because some  files have 2 columns for years and some have only one.  
    try:
      temp = pd.DataFrame(l3,columns=['zero','one','two'])
    except:
      temp = pd.DataFrame(l3,columns=['zero','one'])
    
    # Replace all empty cells with nan as the evaluator dosen't accept NaN
    temp.replace(to_replace=np.nan,value='nan',inplace=True)

    # Most of the files 'STATEMENTS' OR '®' as ending to the indormation that we require. So we remove the extra part.(Better done using try : except block) 
    try:
      statement_index = int(temp[temp['zero']=='STATEMENTS'].index.values)
      temp = temp[:statement_index]
    except:
      statement_index = int(temp[temp['zero']=='®'].index.values[0])
      temp = temp[:statement_index]
    
    # Choosing what column to drop when we have 2 columns with 2019 and other years.
    our_req = 2019
    year = ''
    try:
      if str(our_req) == str(temp['one'].iloc[0]):
        year = 'two'
      elif str(our_req) == str(temp['two'].iloc[0]):
        year='one'
      
      # Remove all whitespaces from the cells of the dataframe.
      temp = temp.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

      # Convert/Substitute all the numbers with commas to non-comma seperated numbers and remove brackets and place negative symbol, as per the requirement.
      temp[temp.columns[1]] = temp[temp.columns[1]].replace(to_replace = r'[^]nan[^\(?\d\.\)?$]',value='',regex=True)
      temp[temp.columns[1]] = temp[temp.columns[1]].apply(check_para)

      # Drop the column that we don't require
      temp.drop(year,axis=1,inplace=True)

      # Create the dictionary format that the submission is to be done.
      temp_dict = dict(zip(temp['zero'][2:],temp[temp.columns[1]][2:]))

    except:
      #This block is executed when there is only one column for year in our data and it has 2019 in it. And makes a dictionary.
      if str(our_req) == str(temp['one'].iloc[0]):
        temp_dict = dict(zip(temp['zero'][2:],temp[temp.columns[1]][2:]))

      else:
        # This block will be executed when the data has only one column but dosen't have 2019 in it. The values are replaces with nan as perthe requirements.
        temp_dict = dict(zip(temp['zero'][2:],temp[temp.columns[1]][2:]))
        temp_dict = dict.fromkeys(temp_dict,'nan')

    temp_dict = str(temp_dict)
    values.append(temp_dict)

  except Exception as e:
    # This block takes in some exception in files like '-' in the data and such (total 65 cases)
    values.append('nan')
    continue

100%|██████████| 500/500 [00:03<00:00, 138.97it/s]


In [149]:
# Count all the nan's that came from the exception.
print(values.count('nan'))

65

In [129]:
print(values[:10])

["{'Current Assets': '312', 'Creditors: amounts falling due within one year': '-59', 'Net current assets (liabilities)': '253', 'Total assets less current liabilities': '253', 'Total net assets (liabilities)': '253', 'Capital and reserves': '253'}",
 "{'Current assets': 'nan', 'Cash at bank and in hand': '1000', 'Net assets': '1000', 'Issued share capital': 'nan', '1000 Ordinary Shares 1 of &#163 each': '1000', 'Total Shareholder funds': '1000'}",
 'nan',
 "{'Current Assets': '23349', 'Creditors: amounts falling due within one year': '-118053', 'Net current assets (liabilities)': '-94704', 'Total assets less current liabilities': '-94704', 'Total net assets (liabilities)': '-94704', 'Capital and reserves': '-94704'}",
 "{'Called up share capital not paid': '1000', 'Current Assets': '4240', 'Creditors: amounts falling due within one year': '-3082', 'Net current assets (liabilities)': '1158', 'Total assets less current liabilities': '2158', 'Total net assets (liabilities)': '2158', 'Capi

In [140]:
# replace utf encoded symbols to html escape codes

for j in tqdm(range(len(values)),position=0,leave=True):
  
  if values[j]=='nan':
    continue

  else:
    # replace all the utf-encoded currency symbol with its equivalent HTML escape code as per the requirements.
    for i in values[j]:
      if i in ['$','®','<','>', '¢', '£', '¥', '€', '§', '©','₹']:
        values[j] = values[j].replace(i,html.escape(i).encode('ascii', 'xmlcharrefreplace').decode('utf-8')[:-1])


100%|██████████| 500/500 [00:00<00:00, 14145.48it/s]


In [0]:
# this code snippet uses json.dumps() to get the required format for submission.
values1 = []
for i in values:
  if i == 'nan':
    values1.append('nan')
  else:
    values1.append(json.dumps(eval(i)))

In [142]:
# Create the submission dataframe and give appropriate headers (the second column header must be given a space b/w extracted and values manually in the csv file).
sub = pd.DataFrame(dict(Filename=names,ExtractedValues=values1))

,Filename,ExtractedValues
0,X8XUG1JD,"{""Current Assets"": ""312"", ""Creditors: amounts ..."
1,X8XX000W,"{""Current assets"": ""nan"", ""Cash at bank and in..."
2,X8XX00J6,"{""Called up share capital not paid"": ""0"", ""Fix..."
3,X8XX00LL,"{""Current Assets"": ""23349"", ""Creditors: amount..."
4,X8XX00YW,"{""Called up share capital not paid"": ""1000"", ""..."
...,...,...
495,X8Y4WHHN,"{""Current assets"": ""nan"", ""Cash at bank and in..."
496,X8Y4WJGP,"{""Current assets"": ""nan"", ""Cash at bank and in..."
497,X8Y4WKMB,nan
498,X8Y4WHBC,"{""Called up share capital not paid"": ""0"", ""Cur..."


In [0]:
sub.to_csv('/content/drive/My Drive/HCL hiring challenge/HCL ML Challenge/eleventh_formatted.csv',index=False)

# ignore this is just code to download the csv file from colab.
  # from google.colab import files
  # files.download('/content/drive/My Drive/HCL hiring challenge/HCL ML Challenge/eleventh_formatted.csv')